# 🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Auteur : Thibault Launois*

*Date : 26/10/2025*

**Description :**

Ce script charge les données de **List_iti_D4G_isere_output_altitude.csv** généré par **2025-10-20_TLaunois_C2C_altitude.ipynb** et va créer un geoparquet.
Les points de départ pour chaque topo n'étant pas fourni par C2C, on définit le point de départ comme étant le point d'un topo ayant l'altitude la plus basse.

# Importation des modules

In [1]:
import os

import geopandas as gpd
import pandas as pd

# working directory is the root of the project
os.chdir("../../")
from src.settings import EPSG_WGS84

# Charger les données

In [2]:
df = pd.read_csv("src/data/C2C/Liste_iti_D4G_isere_output_altitude.csv", index_col=0)
df.head(2)

,Titre itinéraire,Id itinéraire,url,activité,durée,dénivelé (m),longueur (m),Région/massif,Id wp principal,"(id WP - titre - [X,Y] - accessibilité (si renseigné) )",X,Y,lon,lat,altitude
0,Les Bans:Spigolo Sud du Contrefort Médian. Voi...,1780070,https://www.camptocamp.org/routes/1780070,mountain_climbing,['1'],NaN,0.0,(14328 - Isère)(14403 - Écrins),39056.0,"(39056 - Les Bans - [705350.3499286951, 55976...",705350.349929,5.597690e+06,6.336270,44.848420,3564.832275
1,Les Bans:Spigolo Sud du Contrefort Médian. Voi...,1780070,https://www.camptocamp.org/routes/1780070,mountain_climbing,['1'],NaN,0.0,(14328 - Isère)(14403 - Écrins),39056.0,(127458 - Entre Les Aygues - [712692.092985492...,712692.092985,5.594929e+06,6.402222,44.830833,1613.258789


# Déterminer le point de départ pour chaque topo

On fait l'hypothèse que le point de départ d'un topo est le point ayant l'altitude la plus basse.\
La colonne "Id Itinéraire" permet de compter le nombre de topo dans la base de donnée, attention différents topos peuvent avoir le même nom.

In [3]:
# je crée le dataframe avec le point de départ pour chaque topo
data = []

for id in df["Id itinéraire"].unique():
    data.append(list(df[df["Id itinéraire"] == id].sort_values("altitude").iloc[0]))

# aws = access way point
df_awp = pd.DataFrame(data=data, columns=df.columns)

# Rajout des colonnes avec l'id du point du départ et le nom
Dans la colonne **(id WP - titre - [X,Y] - accessibilité (si renseigné) )**, on a les informations du point de départ.

In [4]:
column_name = "(id WP - titre - [X,Y] - accessibilité (si renseigné) )"
regex_pattern = r"(?:\()(\d+)(?:\s-\s+)(.+)(?=\s-\s\[\d)"

df_awp[["Id wp", "Name wp"]] = df_awp[column_name].str.split(
    pat=regex_pattern, regex=True, expand=True
)[[1, 2]]

# strip whitespaces on the name
df_awp["Name wp"] = df_awp["Name wp"].str.strip()
# transform the id from str to int
df_awp["Id wp"] = df_awp["Id wp"].astype(int)

# on regroupe et compte le nombre de topo par point de départ
df_awp2 = (
    df_awp.groupby(["Id wp", "Name wp", "lat", "lon"]).nunique()["Id itinéraire"].to_frame()
)
df_awp2 = df_awp2.reset_index()

In [5]:
# df_awp2.to_csv("../data/C2C/Liste_point_depart_isere.csv", index = False)

# Création du geo-parquet
Il faut renommer les colonnes afin de coller au format pour le géoparquet

In [6]:
# Je renomme les colonnes existantes avec le bon nom
df_awp2 = df_awp2.rename(
    columns={
        "Name wp": "name",
        "Id wp": "navitia_id",
        "Id itinéraire": "nombre_de_depart_de_topo",
    }
)

In [7]:
# je crée les colonnes inexistantes et je les rempli de vide
df_awp2["description"] = ""
df_awp2["geometry"] = ""

In [8]:
# je transforme la colonne latitude et longitude en points (lon,lat) en créant le geo-dataframe
gdf = gpd.GeoDataFrame(
    df_awp2,
    geometry=gpd.points_from_xy(df_awp2.lon, df_awp2.lat, crs=EPSG_WGS84),
)

In [9]:
gdf.head()

,navitia_id,name,lat,lon,nombre_de_depart_de_topo,description,geometry
0,38440,Pierre Blanche,44.889438,5.523988,3,,POINT (5.52399 44.88944)
1,39041,Alpe d'Huez - Falaises du lac Besson,45.117274,6.094444,5,,POINT (6.09444 45.11727)
2,39108,Les Arias d'en bas (Mariande),44.917030,6.176160,1,,POINT (6.17616 44.91703)
3,39178,Cirque inférieur du Boulon,45.191281,5.961359,1,,POINT (5.96136 45.19128)
4,39195,Le Bourg-d'Oisans - Commères,45.027200,6.081170,11,,POINT (6.08117 45.0272)


In [10]:
# Je réordonne les colonnes dans le bon ordre
gdf = gdf[
    [
        "navitia_id",
        "name",
        "description",
        "geometry",
        "nombre_de_depart_de_topo",
    ]
]

In [11]:
# je sauvegarde au format dataframe
gdf.to_parquet("src/data/C2C/depart_topos_stops_isere.parquet", index=False)